<a href="https://colab.research.google.com/github/Zfeng0207/FIT3199-FYP/blob/dev%2Fzfeng/model_inferencing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import os
os.chdir('/content/drive/MyDrive/Colab Notebooks (1)/ECG-MIMIC-main')

In [ ]:
import os
print(os.getcwd())

/content/drive/.shortcut-targets-by-id/1rFfT7F_iTUUjh7EWzWgquzPn3Fzd6KTD/Colab Notebooks/ECG-MIMIC-main


In [ ]:
import os
print(os.listdir())

['LICENSE', 'README.md', 'MIMIC-IV-ECG zip file', 'src', 'reports', 'myenv', 'pretrained_ecg_encoder.pth', 'mimic-iv-2.2', 'saved_models', 'augmentation_examples.png', 'debug_report.txt', 'ecg_model_results', 'ecg_stroke_results', 'ecg_stroke_ensemble', 'enhanced_ecg_results', 'mlruns', 'lightning_logs', 'checkpoints', 'patient_1_12lead_ecg.png', 'patient_3_12lead_ecg.png', 'patient_2_12lead_ecg.png', '__pycache__', 'model.py', 'rnn_attention_model.py', 'stroke_predictions.csv']


In [ ]:
# Step 3: Try importing again
!pip install pytorch_lightning

In [ ]:
import rnn_attention_model
print(dir(rnn_attention_model))

['BinaryAUROC', 'BinaryAccuracy', 'BinaryF1Score', 'CNN', 'ConvNormPool', 'F', 'LSTMClassifier', 'RNN', 'RNNAttentionModel', 'RNNModel', 'Swish', '__all__', '__builtins__', '__cached__', '__doc__', '__file__', '__loader__', '__name__', '__package__', '__spec__', 'nn', 'pad_sequence', 'pl', 'safe_collate', 'torch']


In [ ]:
from rnn_attention_model import RNNAttentionModel, ConvNormPool, Swish, RNN, CNN

In [ ]:
import torch
import numpy as np
import pandas as pd

torch.serialization.add_safe_globals([RNNAttentionModel, ConvNormPool])
# Load model
model = torch.load("src/data/full_model.pkl", map_location='cpu', weights_only=False)
model.eval()

# Load data
df = pd.read_csv("src/data/label_df.csv")
print(df.columns)
memmap_path = "src/data/memmap/memmap.npy"
memmap_data = np.memmap(memmap_path, dtype=np.float32, mode='r')

results = []

for idx in range(len(df)):
    subject_id = df.loc[idx, 'study_id']
    start = df.loc[idx, 'start']
    length = df.loc[idx, 'length']

    # Skip invalid lengths
    if length <= 0:
        continue

    # Extract signal
    raw_signal = memmap_data[start: start + length * 12]
    if len(raw_signal) != length * 12:
        continue  # Incomplete signal

    # Normalize and reshape
    normalized = (raw_signal - raw_signal.mean()) / (raw_signal.std() + 1e-6)
    signal = normalized.reshape(length, 12)
    if np.isnan(signal).any() or np.isinf(signal).any():
        continue

    # Predict
    input_tensor = torch.tensor(signal, dtype=torch.float32).unsqueeze(0)
    with torch.no_grad():
        logits = model(input_tensor)
        prob = torch.sigmoid(logits).item()
        pred = 'Y' if prob > 0.5 else 'N'

    results.append((subject_id, pred))

# Convert to DataFrame
predictions_df = pd.DataFrame(results, columns=['study_id', 'stroke'])

# Save or print
print(predictions_df.head())
predictions_df.to_csv("stroke_predictions.csv", index=False)


Index(['Unnamed: 0', 'filename', 'study_id', 'patient_id', 'ecg_time',
       'label_train', 'all_diag_all', 'label_stroke', 'start', 'length', 'res',
       'stroke_yn'],
      dtype='object')
